# Create Driver Summary (Quantitative Analysis)
- This notebook take about 5 to 10 minutes to execute
- Extract quantitative data from the Trips data
  - How many pick up locations
  - Do they drive to the airport
  - Do they cross state lines
  - Do they work only certain days of the week
- Create a LLM summary of the extracted data

## Create Summary Prompt and run through LLM

In [ ]:
%%bigquery

-- OPTIONAL: Reset all the fields to null
-- If you need to reset you data back to fresh data run the stored procedure: CALL `${project_id}.${bigquery_rideshare_llm_curated_dataset}.sp_reset_demo`();

/*
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver`
   SET driver_quantitative_analysis_prompt     = NULL,
       llm_driver_quantitative_analysis_json   = NULL,
       llm_driver_quantitative_analysis        = NULL
 WHERE TRUE;
*/

In [ ]:
%%bigquery

-- Create the LLM prompt
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
   SET driver_quantitative_analysis_prompt =
       CONCAT('Write a 3 to 8 sentence summary of the following attributes of a driver in third person gender neutral form: ',
               CASE WHEN pickup_location_habit = 'few-1-3-pickup-locations' AND pickup_location_count = 1
                                                                                THEN CONCAT('- The driver only picks up customers at ',
                                                                                             CAST(pickup_location_count AS STRING),
                                                                                             ' pickup location.  These location is ',
                                                                                             distinct_pickup_location_zones,
                                                                                             '.\n')
                    WHEN pickup_location_habit = 'few-1-3-pickup-locations'     THEN CONCAT('- The driver only picks up customers at ',
                                                                                             CAST(pickup_location_count AS STRING),
                                                                                             ' pickup locations.  These locations are: ',
                                                                                             distinct_pickup_location_zones,
                                                                                             '.\n')
                    WHEN pickup_location_habit = 'average-4-6-pickup-locations'     THEN CONCAT('- The driver picks up customers at ',
                                                                                             CAST(pickup_location_count AS STRING),
                                                                                             ' pickup locations.  This is an average number. These locations are: ',
                                                                                             distinct_pickup_location_zones,
                                                                                             '.\n')
                    WHEN pickup_location_habit = 'many-7-9-pickup-locations'     THEN CONCAT('- The driver picks up customers many pickup locations. ',
                                                                                             'This is an above average number. These locations include: ',
                                                                                             distinct_pickup_location_zones,
                                                                                             '.\n')
                    WHEN pickup_location_habit = 'any-pickup-locations'         THEN 'The driver will pickup customers at a large number of locations.'
                    ELSE ''
               END,
               CASE WHEN dropoff_location_habit = 'few-1-3-dropoff-locations' AND dropoff_location_count = 1
                                                                                THEN CONCAT('- The driver only drops off customers at ',
                                                                                             CAST(dropoff_location_count AS STRING),
                                                                                             ' dropoff location.  The location is ',
                                                                                             distinct_dropoff_location_zones,
                                                                                             '.\n')
                    WHEN dropoff_location_habit = 'few-1-3-dropoff-locations'     THEN CONCAT('- The driver only drops off customers at ',
                                                                                             CAST(dropoff_location_count AS STRING),
                                                                                             ' dropoff locations.  These locations are: ',
                                                                                             distinct_dropoff_location_zones,
                                                                                             '.\n')
                    WHEN dropoff_location_habit = 'average-4-6-dropoff-locations'     THEN CONCAT('- The driver drops off customers at ',
                                                                                             CAST(dropoff_location_count AS STRING),
                                                                                             ' dropoff locations.  This is an average number. These locations are: ',
                                                                                             distinct_dropoff_location_zones,
                                                                                             '.\n')
                    WHEN dropoff_location_habit = 'many-7-9-dropoff-locations'     THEN CONCAT('- The driver drops off customers many  locations. ',
                                                                                             'This is an above average number. These locations include: ',
                                                                                             distinct_dropoff_location_zones,
                                                                                             '.\n')
                    ELSE ''
               END,

               CASE WHEN cross_state_habit = 'crosses-state-line' THEN CONCAT('- The driver is will to pickup or dropoff customers accross state lines.\n')
                    WHEN cross_state_habit = 'does-not-cross-state-line' THEN CONCAT('- The driver is not willing to drive accross state lines.\n')
                    ELSE ''
                END,

               CASE WHEN airport_habit = 'airport-driver' THEN CONCAT('- The driver has a high preference for picking and dropping off customers at the airport.\n')
                    WHEN airport_habit = 'non-airport-driver' THEN CONCAT('- The driver typically does not pickup or dropoff at the airport.\n')
                    ELSE ''
                END,

                CASE WHEN day_of_week = 'weekend-driver' THEN CONCAT('- The driver only works on weekends.\n')
                     WHEN day_of_week = 'weekday-driver' THEN CONCAT('- The driver only works on weekdays.\n')
                    ELSE '- The driver works a varity of days not targetting specific days of the week.\n'
                END,

                CASE WHEN hour_of_day = 'night-hour-driver' THEN CONCAT('- The driver likes to work late at night.\n')
                     WHEN hour_of_day = 'rush-hour-driver'  THEN CONCAT('- The driver likes to work a split shift which appears to target rush hour.\n')
                    ELSE '- The driver does not appear to have a set schedule for the hours they work.\n'
                END,

                CASE WHEN average_daily_pay IS NOT NULL THEN CONCAT('- The driver appears to target a specific amount of income per day.\n',
                                                                    '- The drivers likes to their daily amount to be approximately $',
                                                                    CAST(ROUND(average_daily_pay,2) AS STRING),
                                                                    ' with a ',
                                                                    CAST(ROUND(stddev_amt,2) AS STRING),
                                                                    '% standard deviation.\n')
                    ELSE ''
                END
             )
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver_quantitative_analysis` AS driver_quantitative_analysis
WHERE driver.driver_id = driver_quantitative_analysis.driver_id
;



In [ ]:
%%bigquery

SELECT driver_quantitative_analysis_prompt
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
 WHERE driver_quantitative_analysis_prompt IS NOT NULL
 LIMIT 10;

## Run the LLM to generate a Driver Summary on Quantitative Analysis

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

In [ ]:
# Process in batches
batch_size = 100

# Set the parameters so we are more deterministic and less creative/random responses
llm_temperature = .80
llm_max_output_tokens = 1024
llm_top_p = .70
llm_top_k = 25

update_sql="""
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
  SET llm_driver_quantitative_analysis_json = child.ml_generate_text_result
  FROM (SELECT *
          FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_enriched_dataset}.gemini_model`,
              (SELECT driver_id,
                      driver_quantitative_analysis_prompt AS prompt
                FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver`
                WHERE (llm_driver_quantitative_analysis_json IS NULL
                       OR
                       JSON_VALUE(llm_driver_quantitative_analysis_json, '$.candidates[0].content.parts[0].text') IS NULL
                       )
                  AND include_in_llm_processing = TRUE
                  AND driver_quantitative_analysis_prompt IS NOT NULL
                LIMIT {batch_size}),
              STRUCT(
                {llm_temperature} AS temperature,
                {llm_max_output_tokens} AS max_output_tokens,
                {llm_top_p} AS top_p,
                {llm_top_k} AS top_k
                ))
  ) AS child
WHERE driver.driver_id = child.driver_id
  """.format(batch_size = batch_size,
             llm_temperature = llm_temperature,
             llm_max_output_tokens = llm_max_output_tokens,
             llm_top_p = llm_top_p,
             llm_top_k = llm_top_k)

print("SQL: {update_sql}".format(update_sql=update_sql))


In [ ]:
# Score while records remain
# score in groups of batch_size records (we can do up to 10,000 at a time)
import time

done = False
displayed_first_sql = False
original_record_count = 0

while done == False:
  # Get the count of records to score
  sql = """
        SELECT COUNT(*) AS cnt
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver`
         WHERE (llm_driver_quantitative_analysis_json IS NULL
                OR
                JSON_VALUE(llm_driver_quantitative_analysis_json, '$.candidates[0].content.parts[0].text') IS NULL
                )
           AND include_in_llm_processing = TRUE
           AND driver_quantitative_analysis_prompt IS NOT NULL;
        """

  df_record_count = client.query(sql).to_dataframe()
  cnt = df_record_count['cnt'].head(1).item()
  if displayed_first_sql == False:
    original_record_count = cnt
    displayed_first_sql = True

  print("Remaining records to process: ", cnt, " out of", original_record_count, " batch_size: ", batch_size)


  if cnt == 0:
    done = True
  else:
    # https://github.com/googleapis/python-bigquery/tree/master/samples
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(update_sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))

    while query_job.state != "DONE":
      time.sleep(5)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))


## Parse the LLM JSON results

In [ ]:
%%bigquery

UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` driver
   SET llm_driver_quantitative_analysis = JSON_VALUE(llm_driver_quantitative_analysis_json, '$.candidates[0].content.parts[0].text')
 WHERE llm_driver_quantitative_analysis_json IS NOT NULL
   AND llm_driver_quantitative_analysis IS NULL;

In [ ]:
%%bigquery

SELECT driver_id, driver_quantitative_analysis_prompt, llm_driver_quantitative_analysis_json, llm_driver_quantitative_analysis
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver`
 WHERE llm_driver_quantitative_analysis_json IS NOT NULL
   AND driver_quantitative_analysis_prompt IS NOT NULL
LIMIT 20;
